In [88]:
import pandas as pd
import os

# Function to convert the state code and date to year and month
def convert_to_date(row):
    state_code = row['data'][:3]
    date = row['data'][3:]
    year = date[-4:]
    month = row['month'].split('_')[-1]  # Extracting the month number from the 'month' column
    return pd.Series([state_code, year, month, row['temp']], index=['stateCode', 'year', 'month', 'temperature'])

# List to store individual DataFrames
dfs = []
data_directory = "./Desktop/data-vis/assignment_3_preprocessing"

txt_files = [file for file in os.listdir(data_directory) if file.endswith(".txt")]

# Process each file
for file_name in txt_files:
    file_path = os.path.join(data_directory, file_name)
    print("processing ..." + file_path)
    
    columns = ['data'] + [f'temp_{i}' for i in range(1, 13)]
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns, dtype={'data': str})

    # Reshape the data using melt to have a single column for temperatures
    df = pd.melt(df, id_vars=['data'], value_vars=[f'temp_{i}' for i in range(1, 13)], var_name='month', value_name='temp')

    # Apply the conversion function to create new columns
    df[['stateCode', 'year', 'month', file_name.split('_')[0]]] = df.apply(convert_to_date, axis=1)

    # Drop unnecessary columns
    df = df[['stateCode', 'year', 'month', file_name.split('_')[0]]]

    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Print the final DataFrame


processing ..../Desktop/data-vis/assignment_3_preprocessing/min_data.txt
processing ..../Desktop/data-vis/assignment_3_preprocessing/max_data.txt
processing ..../Desktop/data-vis/assignment_3_preprocessing/average_data.txt


In [90]:
print(dfs[0].sort_values(by=['stateCode', 'year']).head(1))
print(dfs[1].sort_values(by=['stateCode', 'year']).head(1))
print(dfs[2].sort_values(by=['stateCode', 'year']).head(1))


  stateCode  year month   min
0       001  1895     1  33.4
  stateCode  year month   max
0       001  1895     1  52.7
  stateCode  year month  average
0       001  1895     1     43.1


In [98]:
merged_df_temp = pd.merge(dfs[0], dfs[1], on=['stateCode', 'year', 'month'])
merged_df = pd.merge(merged_df_temp, dfs[2], on=['stateCode', 'year', 'month'])
merged_df = merged_df.sort_values(by=['stateCode', 'year'])

In [101]:
merged_df.to_csv(data_directory + "/merged_results.csv", encoding='utf-8', index=False)


In [102]:
dfs[0].sort_values(by=['stateCode', 'year']).to_csv(data_directory + "/min.csv", encoding='utf-8', index=False)
dfs[1].sort_values(by=['stateCode', 'year']).to_csv(data_directory + "/max.csv", encoding='utf-8', index=False)
dfs[2].sort_values(by=['stateCode', 'year']).to_csv(data_directory + "/average.csv", encoding='utf-8', index=False)

In [114]:

# Group by 'stateCode' and 'year', then calculate the average of 'min' and 'max'
result_df = merged_df.groupby(['stateCode', 'year']).agg({'min': 'mean', 'max': 'mean'}).round(2).reset_index()
result_df.to_csv(data_directory + "/ridge_line.csv", encoding='utf-8', index=False)
# Print the result DataFrame
print(result_df.head(20))    

   stateCode  year    min    max
0        001  1895  50.21  73.07
1        001  1896  52.77  75.78
2        001  1897  52.46  75.92
3        001  1898  51.73  74.22
4        001  1899  51.33  74.90
5        001  1900  51.99  74.81
6        001  1901  49.19  73.58
7        001  1902  51.64  75.52
8        001  1903  50.26  73.69
9        001  1904  50.22  75.32
10       001  1905  51.32  73.96
11       001  1906  51.41  75.12
12       001  1907  51.78  75.63
13       001  1908  51.75  76.33
14       001  1909  51.42  76.30
15       001  1910  50.19  75.12
16       001  1911  53.33  77.31
17       001  1912  50.55  73.66
18       001  1913  51.46  75.77
19       001  1914  50.88  74.75
